In [17]:
from PIL import Image, ImageOps
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import SGDClassifier

In [20]:
def aanmakenDf (groep, label):
#Drie lege lijsten worden aangemaakt die dan uiteindelijk samen dataframe zullen vormen 
    Pixels= []
    Label =[]
    Naam=[]
  
#fotos inlezen en omzetten per groep. 
    for foto in os.listdir(groep):
        beeld = Image.open(groep+foto)
#fotos omzetten in grijswaarde of 2 dimensionele array ipv 3x2
        beeld_grijs = ImageOps.grayscale(beeld)

#fotos resizen voor leesbaarheid tijdens testen, en eventueel voor latere experimenten
        beeld_formaat = beeld_grijs.resize((beeld_grijs.width // 1, beeld_grijs.height //1 ))       
        
#fotos omzetten naar numpy array
        data_2dim = np.asarray(beeld_formaat, dtype=np.uint8)
        data = data_2dim.flatten()

#3 lijsten opvullen, eerste met data van foto, tweede met label en derde met naam van foto. 
#De lengte van elke lijst = aantal fotos in bijhorende map
        Pixels.append(data)
        Label.append(label)
        Naam.append(foto)

#Per groep worden de drie aangemaakte lijsten omgezet naar panda df
    dict = {'data': Pixels, 'label': Label, 'naam': Naam}
    df = pd.DataFrame(dict)
    return df

In [21]:
#Dataframe aanmaken
aanwezig = r"classificatie/aanwezig/"  #852 foto's 
buiten = r"classificatie/buiten/"       #389 foto's
niets = r"classificatie/niets/"         #1399 foto's
#Eerst wordt per map een panda dataframe aangemaakt
nietsDf = aanmakenDf(niets, 0)
aanwezigDf = aanmakenDf(aanwezig, 1)
buitenDf = aanmakenDf(buiten, 2)
#vervolgens worden ze alle drie samengevoegd tot 1 groot dataframe
volledigDataframe = pd.concat([nietsDf, aanwezigDf, buitenDf], ignore_index=True)

In [4]:
#Opsplitsen in gestratificieerde testset en trainingsset
np.random.seed(42)
split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(volledigDataframe, volledigDataframe['label']):
    strat_train_set = volledigDataframe.loc[train_index]
    strat_test_set = volledigDataframe.loc[test_index] 
#X = data, y = label. Dit voor trainingsset en testset  
X_train, X_test, y_train, y_test, naam_train, naam_test = strat_train_set['data'], strat_test_set['data'], strat_train_set['label'], strat_test_set['label'], strat_train_set['naam'], strat_test_set['naam']

In [5]:
#Omzetten naar np.array
X_train_array = np.array(X_train.tolist())
X_test_array = np.array(X_test.tolist())
y_train_array = np.array(y_train)
y_test_array = np.array(y_test)
naam_train_array= np.array(naam_train)
naam_test_array= np.array(naam_test)

In [6]:
#RandomforestClassifier
from sklearn.ensemble import RandomForestClassifier
forest_clf = RandomForestClassifier(random_state=42)
forest_clf.fit(X_train_array, y_train_array)

RandomForestClassifier(random_state=42)

In [7]:
#support vector machine model
from sklearn.svm import SVC
svc = SVC(random_state=42)
svc.fit(X_train_array, y_train_array)

SVC(random_state=42)

In [8]:
#One vs one classifier
from sklearn.multiclass import OneVsOneClassifier
ovo_clf = OneVsOneClassifier(SGDClassifier(random_state=42))
ovo_clf.fit(X_train_array, y_train_array)

OneVsOneClassifier(estimator=SGDClassifier(random_state=42))

In [9]:
#Votingclassifier toevoegen? 
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators=[('forest', forest_clf), ('ovo', ovo_clf), ('svc', svc)],voting='hard')
voting_clf.fit(X_train_array, y_train_array)

VotingClassifier(estimators=[('forest',
                              RandomForestClassifier(random_state=42)),
                             ('ovo',
                              OneVsOneClassifier(estimator=SGDClassifier(random_state=42))),
                             ('svc', SVC(random_state=42))])

In [10]:
#Accuraatheid van random forest, oneVSone, supportVectorMachine en voting
from sklearn.metrics import accuracy_score
for clf in (svc, ovo_clf, forest_clf, voting_clf):
 clf.fit(X_train_array, y_train_array)
 y_pred = clf.predict(X_test_array)
 print(clf.__class__.__name__, accuracy_score(y_test_array, y_pred))

SVC 0.9204545454545454
OneVsOneClassifier 0.9318181818181818
RandomForestClassifier 0.9412878787878788
VotingClassifier 0.9337121212121212


In [11]:
#Clustering als preproces toegevoegd dmv pipeline
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

pipeline = Pipeline([
("kmeans", KMeans()),
("forest", RandomForestClassifier()),
])

param_grid = dict(kmeans__n_clusters=range(2, 100))
grid_clf = GridSearchCV(pipeline, param_grid, cv=3, verbose=2)
grid_clf.fit(X_train_array, y_train_array)

grid_clf.best_params_

Fitting 3 folds for each of 98 candidates, totalling 294 fits
[CV] END ...............................kmeans__n_clusters=2; total time=  58.4s
[CV] END ...............................kmeans__n_clusters=2; total time=  56.3s
[CV] END ...............................kmeans__n_clusters=2; total time=  59.0s
[CV] END ...............................kmeans__n_clusters=3; total time= 1.8min
[CV] END ...............................kmeans__n_clusters=3; total time= 1.8min
[CV] END ...............................kmeans__n_clusters=3; total time= 1.7min
[CV] END ...............................kmeans__n_clusters=4; total time= 1.9min
[CV] END ...............................kmeans__n_clusters=4; total time= 1.8min
[CV] END ...............................kmeans__n_clusters=4; total time= 1.9min
[CV] END ...............................kmeans__n_clusters=5; total time= 2.0min
[CV] END ...............................kmeans__n_clusters=5; total time= 2.0min
[CV] END ...............................kmeans_

{'kmeans__n_clusters': 74}

In [12]:
#Accuraatheid van randomForest voorafgegaan door clustering
grid_clf.score(X_test_array, y_test_array)

0.9166666666666666

In [13]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression(random_state=42)
log_reg.fit(X_train_array, y_train_array)

C:\Users\berto\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(random_state=42)

In [14]:
log_reg.score(X_test_array, y_test_array)

0.9375

In [15]:
pipeline2 = Pipeline([
("kmeans", KMeans(n_clusters=74)),
("log_reg", LogisticRegression()),
])
pipeline2.fit(X_train_array, y_train_array)

C:\Users\berto\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('kmeans', KMeans(n_clusters=74)),
                ('log_reg', LogisticRegression())])

In [16]:
pipeline2.score(X_test_array, y_test_array)

0.8712121212121212